In [1]:
import numpy as np
import netCDF4
import h5netcdf
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import pickle
from os.path import join
sys.path.append('/home/samuel.varga/projects/deep_learning/')

from deep_learning.training_utils import load_rotation, convert_to_tf, resize_neural_net
from deep_learning.deep_networks import create_U_net_classifier_2D

2023-12-29 16:22:13.758835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 16:22:13.830001: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from keras import backend as K
import gc
from numba import cuda
outdir='/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/'

In [3]:
batch_size=2048
target_column='any_severe__36km'
rotation=0
X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data'), rotation, target_column)
X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data'), rotation, target_column)
X_test, y_test = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__testing_data'), None, target_column)
print(np.shape(X_train))
print(np.shape(y_train))

#Convert to tf dataset
train_ds = convert_to_tf((X_train,np.expand_dims(y_train, axis=-1)), batch_size)
val_ds = convert_to_tf((X_val, np.expand_dims(y_val, axis=-1)), batch_size)
test_ds=convert_to_tf((X_test[None,:,:,:], np.expand_dims(y_test[None,:,:,:], axis=-1)))

Training path detected - loading scaling
(6350, 16, 16, 63)
(6350, 16, 16)


2023-12-29 16:22:19.387041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 16:22:19.528012: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79086 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:21:00.0, compute capability: 8.0


In [4]:
#U-net architectural parameters
i=2
conv_filters=[i*n for n in [32,64,128,256]]
conv_size=[4,3,2,2]
max_pool=[2,2,2,2]
conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                   else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                   for s, f, p, in zip(conv_size, conv_filters, max_pool)]
args={'lrate':1e-4, 'loss':'binary_focal_crossentropy','activation_conv':'relu','activation_out':'sigmoid',
     'p_spatial_dropout':0.1, 'filters':conv_filters, 'size':conv_size, 'pool':max_pool, 'shape':(16,16),
     'rotation':rotation,'target_column':target_column, 'i':i}
thresholds= [0.15]
metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.MeanSquaredError(name='Brier score'),
    tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.AUC(name='prc', curve='PR'),     
         tf.keras.metrics.FalseNegatives(thresholds=thresholds), tf.keras.metrics.FalsePositives(thresholds=thresholds),
         tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.TrueNegatives(thresholds=thresholds),
         tf.keras.metrics.TruePositives(thresholds=thresholds)]

In [182]:
#Create U-net
u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=1, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                               lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out='sigmoid',
                                      normalization=(mean, variance))

Model: "model_30"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_30 (Normalizatio  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 n)                                                                                               
                                                                                                  
 Encode_0_0 (Conv2D)            (None, 16, 16, 64)   64576       ['normalization_30[0][0]']       
                                                                                           

 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 256)    262400      ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Upsample_1 (UpSampling2  (None, 8, 8, 256)   0           ['De_Sp_Dr_2_0[0][0]']           
 D)                                                                                               
          

In [183]:
#Callbacks
early_stopping_cb =keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True,
                                                min_delta=0.001, monitor='val_loss')
tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,'logs'), histogram_freq=1)

In [184]:
#Learn the model
history = u_net.fit(train_ds, epochs=5, verbose=True, validation_data = val_ds,
    callbacks=[early_stopping_cb, tensorboard_cb])

Epoch 1/5


2023-12-29 15:57:04.693861: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_30/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 [==============================] - 2s 277ms/step - loss: 0.1233 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.1868 - auc: 0.3419 - prc: 0.0337 - false_negatives_31: 5113.0000 - false_positives_31: 1541759.0000 - mean_absolute_error: 0.4217 - true_negatives_31: 8271.0000 - true_positives_31: 70457.0000 - val_loss: 0.0750 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.1184 - val_auc: 0.4729 - val_prc: 0.0433 - val_false_negatives_31: 985.0000 - val_false_positives_31: 356360.0000 - val_mean_absolute_error: 0.3200 - val_true_negatives_31: 16817.0000 - val_true_positives_31: 17518.0000
Epoch 2/5
4/4 [==============================] - 1s 187ms/step - loss: 0.0775 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.1105 - auc: 0.4647 - prc: 0.0424 - false_negatives_31: 11325.0000 - false_positives_31: 1376332.0000 - mean_absolute_error: 0.3013 - true_negatives_31: 173698.0000 - true_positives_31: 64245.0000 - val_loss: 0.0544 - val_sparse_categorical_accuracy:

In [12]:
def save_results(args, u_net, history, train_ds, val_ds, test_ds):
    fbase = f"{args['target_column']}_Rot_{args['rotation']}_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_i_{args['i']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}_loss_{args['loss']}"
    results = {}
    results['args'] = args
    #results['predict_val'] = u_net.predict(val_ds)
    results['predict_val_eval'] = u_net.evaluate(val_ds)

    if test_ds is not None:
        #results['predict_test']=u_net.predict(test_ds)
        results['predict_test_eval']=u_net.evaluate(test_ds)

    #results['predict_train']=u_net.predict(train_ds)
    results['predict_train_eval']=u_net.evaluate(train_ds)
    results['history']=history.history
    results['fname_base']=fbase

    #Save results
    with open(join(join(outdir, 'results'), f'{fbase}_results.pkl'),'wb') as fp:
        pickle.dump(results, fp)

    #save model
    if False:
        u_net.save(join(join(outdir, 'models'), f'{fbase}_model'))
    #print(np.max(results['predict_test']))
    #print(np.mean(results['predict_test']))
    print(fbase)
    return None
save_results(args, u_net, history,  train_ds, val_ds, test_ds)

4/4 [==============================] - 0s 22ms/step - loss: 0.1332 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0369 - auc: 0.8916 - prc: 0.3083 - false_negatives: 26558.0000 - false_positives: 140367.0000 - mean_absolute_error: 0.0775 - true_negatives: 1409663.0000 - true_positives: 49012.0000
any_severe__36km_Rot_0_16_16_lrate_0.0001_spatial_dropout_0.01_i_1_filters_[32, 64, 128, 256]_size_[2, 1, 2, 1]_pool_[2, 2, 2, 2]_loss_binary_crossentropy


In [18]:
#Hyperparam search
from itertools import product
for rotation, p_s, lrate, cs, i, loss in product([0,1], [0.01, 0.1], [0.01, 0.1], ([2,1,2,1],), (1,2), ['binary_crossentropy']):
    X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data'), rotation, target_column)
    X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data'), rotation, target_column)
    print(np.shape(X_train))
    print(np.shape(y_train))
    print(f'Validation Base Rate:{np.mean(y_val)}')
    print(f'Validation non-event Rate:{1-np.mean(y_val)}')
    #Convert to tf dataset
    train_ds = convert_to_tf((X_train,np.expand_dims(y_train, axis=-1)), batch_size)
    val_ds = convert_to_tf((X_val,np.expand_dims(y_val, axis=-1)), batch_size)
    
    #U-net architectural parameters
    conv_filters=[n*i for n in [32,64,128,256]]
    max_pool=[2,2,2,2]
    conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                       else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                       for s, f, p, in zip(cs, conv_filters, max_pool)]
    args={'lrate':lrate, 'loss':loss,'activation_conv':'relu','activation_out':'sigmoid',
         'p_spatial_dropout':p_s, 'filters':conv_filters, 'size':cs, 'pool':max_pool, 'shape':(16,16),
         'rotation':rotation,'target_column':target_column, 'i':i}
##
    #create u-net
    #Pick better loss/activation functions
    u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=1, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                               lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out=args['activation_out'],
                                      normalization=(mean, variance))
##
    early_stopping_cb =keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True,
                                                    min_delta=0.001, monitor='val_loss')
    tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,f'logs/Rot_{rotation}_p_s_{p_s}_lrate_{lrate}_cs_{cs}_i_{i}_loss_{loss}'), histogram_freq=1)
    history = u_net.fit(train_ds, epochs=100, verbose=True, validation_data = val_ds,
        callbacks=[early_stopping_cb, tensorboard_cb])
    
    #save_results(args, u_net, history, train_ds, val_ds, test_ds)

Training path detected - loading scaling
(6350, 16, 16, 63)
(6350, 16, 16)
Validation Base Rate:0.047240093954248365
Validation non-event Rate:0.9527599060457517
Model: "model_278"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_278 (Normalizati  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 on)                                                                                              
                                                                                                  
 Encode_0_0 (Conv2D)       

 Decode_2_1 (Conv2D)            (None, 4, 4, 128)    196736      ['Concat2[0][0]']                
                                                                                                  
 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 128)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 128)    65664       ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 128)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Up

2023-12-29 20:44:17.178490: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_278/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 [==============================] - 2s 177ms/step - loss: 2.4968 - sparse_categorical_accuracy: 0.9558 - Brier score: 0.0874 - auc: 0.5843 - prc: 0.0513 - false_negatives: 36134.0000 - false_positives: 774468.0000 - mean_absolute_error: 0.1879 - true_negatives: 1190379.0000 - true_positives: 54699.0000 - val_loss: 0.2563 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0733 - val_auc: 0.7038 - val_prc: 0.0809 - val_false_negatives: 3899.0000 - val_false_positives: 160981.0000 - val_mean_absolute_error: 0.1871 - val_true_negatives: 212196.0000 - val_true_positives: 14604.0000
Epoch 2/100
4/4 [==============================] - 0s 89ms/step - loss: 0.2422 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0647 - auc: 0.6930 - prc: 0.0822 - false_negatives: 28682.0000 - false_positives: 537556.0000 - mean_absolute_error: 0.1566 - true_negatives: 1012474.0000 - true_positives: 46888.0000 - val_loss: 0.2041 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0

4/4 [==============================] - 0s 88ms/step - loss: 0.1265 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0354 - auc: 0.9053 - prc: 0.3394 - false_negatives: 26909.0000 - false_positives: 122326.0000 - mean_absolute_error: 0.0720 - true_negatives: 1427704.0000 - true_positives: 48661.0000 - val_loss: 0.1383 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0370 - val_auc: 0.8829 - val_prc: 0.3249 - val_false_negatives: 9093.0000 - val_false_positives: 22224.0000 - val_mean_absolute_error: 0.0663 - val_true_negatives: 350953.0000 - val_true_positives: 9410.0000
Epoch 15/100
4/4 [==============================] - 0s 88ms/step - loss: 0.1259 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0352 - auc: 0.9056 - prc: 0.3521 - false_negatives: 27411.0000 - false_positives: 116186.0000 - mean_absolute_error: 0.0706 - true_negatives: 1433844.0000 - true_positives: 48159.0000 - val_loss: 0.1380 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.036

...

4/4 [==============================] - 0s 90ms/step - loss: 0.1880 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0443 - auc: 0.5042 - prc: 0.0469 - false_negatives: 75570.0000 - false_positives: 0.0000e+00 - mean_absolute_error: 0.0901 - true_negatives: 1550030.0000 - true_positives: 0.0000e+00 - val_loss: 0.1902 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0450 - val_auc: 0.5031 - val_prc: 0.0475 - val_false_negatives: 18503.0000 - val_false_positives: 0.0000e+00 - val_mean_absolute_error: 0.0907 - val_true_negatives: 373177.0000 - val_true_positives: 0.0000e+00
Epoch 41/100
4/4 [==============================] - 0s 89ms/step - loss: 0.1880 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0443 - auc: 0.5025 - prc: 0.0467 - false_negatives: 75570.0000 - false_positives: 0.0000e+00 - mean_absolute_error: 0.0901 - true_negatives: 1550030.0000 - true_positives: 0.0000e+00 - val_loss: 0.1902 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.045

 Midway (Conv2D)                (None, 1, 1, 1024)   525312      ['Encode_Pool_3[0][0]']          
                                                                                                  
 Decode_Upsample_3 (UpSampling2  (None, 2, 2, 1024)  0           ['Midway[0][0]']                 
 D)                                                                                               
                                                                                                  
 Concat3 (Concatenate)          (None, 2, 2, 1536)   0           ['Decode_Upsample_3[0][0]',      
                                                                  'En_Sp_Dr_3_1[0][0]']           
                                                                                                  
 Decode_3_1 (Conv2D)            (None, 2, 2, 512)    786944      ['Concat3[0][0]']                
                                                                                                  
 De_Sp_Dr_

2023-12-29 20:45:24.474863: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_281/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 [==============================] - 2s 223ms/step - loss: 192485703811072.0000 - sparse_categorical_accuracy: 0.9534 - Brier score: 0.0999 - auc: 0.5217 - prc: 0.0771 - false_negatives: 67635.0000 - false_positives: 513280.0000 - mean_absolute_error: 0.1754 - true_negatives: 1409927.0000 - true_positives: 26438.0000 - val_loss: 0.5915 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.1994 - val_auc: 0.5000 - val_prc: 0.0472 - val_false_negatives: 0.0000e+00 - val_false_positives: 373177.0000 - val_mean_absolute_error: 0.4458 - val_true_negatives: 0.0000e+00 - val_true_positives: 18503.0000
Epoch 2/100
4/4 [==============================] - 0s 132ms/step - loss: 0.5647 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.1863 - auc: 0.5023 - prc: 0.0475 - false_negatives: 0.0000e+00 - false_positives: 1550030.0000 - mean_absolute_error: 0.4301 - true_negatives: 0.0000e+00 - true_positives: 75570.0000 - val_loss: 0.4851 - val_sparse_categorical_accuracy: 0.9528 - val_Br

KeyboardInterrupt: 

In [7]:
K.clear_session()
gc.collect()
#gpu = cuda.get_current_device()
#gpu.reset()
cuda.close()